In [ ]:
import numpy as np
import math
import cv2


In [ ]:
 #--------CONVOLUTION OPERATIONS------------------------
def flip(k):
    k1=[]
    k=np.asarray(k)
    a=k.shape
    for i in range(len(k)):
        x=[]
        for j in k[a[0]-i-1]:
            x.insert(0,j)
        k1.append(x)
    k1=(np.array(k1))
    return k1

def operation(mat,kernel):
    k=kernel.shape
    sum=0
    for i in range(k[1]):
        for j in range(k[0]):
            try:
                sum=sum+kernel[i,j]*mat[i,j]
            except IndexError:
                print(i,j)
    
    return sum
                        
def convolution(img,kernel):
    kernel=flip(kernel)
    k_rows=kernel.shape[0]
    k_cols=kernel.shape[1]
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    conv_img=np.zeros(img.shape)
    x=int(k_rows/2)
    y=int(k_cols/2)
    padded_image =np.pad(img, ((x,x),(y,y)), 'constant')
    for j in range(img.shape[1]):
        for i in range(img.shape[0]):
            conv_img[i,j]=operation(padded_image[i:i+k_rows,j:j+k_cols],kernel)
    a=conv_img.shape
    x=conv_img[1:a[0]-1,1:a[1]-1]        
    return conv_img
#slicing of the images
def partitions(img,px,py):
    a=img.shape
    vx=int(a[0]/px)
    vy=int(a[1]/py)
    num=[]
    for y in range(py):
        for x in range(px):
            p=img[vx*x:(x+1)*vx, vy*y:(y+1)*vy]
            num.append(p)
    return num

#applying convolution operation to each slice
def apply_filter(img):
    kernel_x=[[-1,0,1],
              [-2,0,2],
              [-1,0,1]]
    kernel_y=[[-1,-2,-1],
              [0,0,0],
              [1,2,1]]
    gradx = abs(convolution(img,kernel_x))+1
    grady = abs(convolution(img,kernel_y))+1
    
    a=img.shape
    
    m=a[0]
    n=a[1]
    mag=np.zeros(shape=(m,n))
    direction=np.zeros(shape=(m,n))
#calculating the magnitude and the direction of the gradient
    for i in range(n):
        for j in range(m):
            mag[i,j]=math.sqrt(gradx[i,j]**2+grady[i,j]**2)
            direction[i,j]=math.degrees(math.atan(grady[i,j]/gradx[i,j]))
#binning process :assigning magnitude of each slice according to the direction 
    bins=np.zeros(shape=(9))
    for i in range(m):
        for j in range(n):
            angle=direction[i,j]
            bin_no=int(angle/20)
            parting_factor=(20-(angle%20))/20
            if bin_no==8:
                bins[8]=bins[8]+parting_factor*mag[i,j]
                bins[0]=bins[0]+(mag[i,j]-parting_factor*mag[i,j])
            else:
                bins[bin_no]=bins[bin_no]+parting_factor*mag[i,j]
                bins[bin_no+1]=bins[bin_no+1]+(mag[i,j]-parting_factor*mag[i,j])
    return bins

